In [2]:
!pip install simpy

In [3]:
import simpy
import random
import csv
from datetime import datetime

# Constants
AVG_ARRIVAL_TIME = 5
NUM_CONTAINERS = 150
CRANE_TIME_PER_CONTAINER = 3
TRUCK_CYCLE_TIME = 6
NUM_BERTHS = 2
NUM_CRANES = 2
NUM_TRUCKS = 3

class ContainerTerminal:
  def __init__(self,env):
       self.env = env
       self.berths = simpy.Resource(env, capacity=NUM_BERTHS)
       self.quay_cranes = simpy.Resource(env, capacity=NUM_CRANES)
       self.trucks = simpy.Resource(env, capacity=NUM_TRUCKS)

class Logger:
  def __init__(self, filename):
    self.filename = filename
    with open(self.filename, 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["Timestamp", "Simulation Time", "Event"])

  def log_event(self, env, message):
    with open(self.filename, 'a', newline='') as file:
      writer = csv.writer(file)
      writer.writerow([datetime.now(), f"{env.now:.2f}", message])
    print(f"{env.now:.2f}: {message}")

def vessel_process(env, terminal, logger, vessel_id):
    containers = NUM_CONTAINERS
    logger.log_event(env, f"Vessel {vessel_id} arrived with {containers} containers")

    with terminal.berths.request() as berth:
        yield berth
        logger.log_event(env, f"Vessel {vessel_id} berthed")

        with terminal.quay_cranes.request() as crane:
            yield crane
            logger.log_event(env, f"Quay crane assigned to Vessel {vessel_id}")

            while containers > 0:
                with terminal.trucks.request() as truck:
                    yield truck
                    yield env.timeout(CRANE_TIME_PER_CONTAINER)  # Time to move container to truck
                    containers -= 1
                    logger.log_event(env, f"Quay crane moved a container from Vessel {vessel_id} to truck. Remaining: {containers}")

                    yield env.timeout(TRUCK_CYCLE_TIME)  # Truck's round trip to yard
                    logger.log_event(env, f"After Dropping Truck returned from yard for Vessel {vessel_id}")

        logger.log_event(env, f"Quay crane released for Vessel {vessel_id}")

    logger.log_event(env, f"Vessel {vessel_id} unloaded and departed. Berth is now free.")


def vessel_generator(env, terminal, logger):
    vessel_id = 1
    while True:
        yield env.timeout(random.expovariate(1 / (AVG_ARRIVAL_TIME * 60)))  # Average 5 hours (300 minutes) between arrivals
        env.process(vessel_process(env, terminal, logger, vessel_id))
        vessel_id += 1

def run_simulation(env, simulation_time, logger):
    terminal = ContainerTerminal(env)
    env.process(vessel_generator(env, terminal, logger))
    logger.log_event(env, f"Starting simulation for {simulation_time} minutes")
    env.run(until=simulation_time)
    logger.log_event(env, "Simulation completed")

if __name__ == "__main__":
    try:
        SIMULATION_TIME = int(input("Enter simulation time in minutes: ")) 
        if SIMULATION_TIME <= 0:
            raise ValueError("Simulation time must be a positive integer.")
    except ValueError as e:
        print(f"Invalid input: {e}")
    else:
        env = simpy.Environment()
        logger = Logger("container_terminal_simulation.csv")
        run_simulation(env, SIMULATION_TIME, logger)



Enter simulation time in minutes: 3000
0.00: Starting simulation for 3000 minutes
49.94: Vessel 1 arrived with 150 containers
49.94: Vessel 1 berthed
49.94: Quay crane assigned to Vessel 1
52.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 149
58.94: After Dropping Truck returned from yard for Vessel 1
61.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 148
67.94: After Dropping Truck returned from yard for Vessel 1
70.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 147
76.94: After Dropping Truck returned from yard for Vessel 1
79.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 146
85.94: After Dropping Truck returned from yard for Vessel 1
88.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 145
94.94: After Dropping Truck returned from yard for Vessel 1
97.94: Quay crane moved a container from Vessel 1 to truck. Remaining: 144
103.94: After Dropping Truck returned from yard for Vessel 1
